In [19]:
import os
import findspark
findspark.init(os.environ['SPARK_HOME'])

from pyspark.sql import SparkSession
from pyspark.sql.functions import lit, when

In [13]:
# Inicializando a sessão do Spark
spark = SparkSession.builder.appName("Exemplo").getOrCreate()

## Data Quality
---

- **Verificação de Integridade Referencial:** Garantir que as relações entre diferentes conjuntos de dados sejam consistentes.


In [14]:
# Criando DataFrames de exemplo para clientes e pedidos
data_clientes = [("Cliente1", 1),
                ("Cliente2", 2),
                ("Cliente3", 3)]

data_pedidos = [("Pedido1", 1),
                ("Pedido2", 2),
                ("Pedido3", 4)]  # Introduzindo uma chave estrangeira inválida

columns_clientes = ["NomeCliente", "IDCliente"]
columns_pedidos = ["DescricaoPedido", "IDCliente"]

df_clientes = spark.createDataFrame(data_clientes, columns_clientes)
df_pedidos = spark.createDataFrame(data_pedidos, columns_pedidos)

In [15]:
df_clientes.show()

+-----------+---------+
|NomeCliente|IDCliente|
+-----------+---------+
|   Cliente1|        1|
|   Cliente2|        2|
|   Cliente3|        3|
+-----------+---------+



In [16]:
df_pedidos.show()

+---------------+---------+
|DescricaoPedido|IDCliente|
+---------------+---------+
|        Pedido1|        1|
|        Pedido2|        2|
|        Pedido3|        4|
+---------------+---------+



In [20]:
# Realizando a verificação de integridade referencial
df_verificado = df_pedidos.join(df_clientes, df_pedidos["IDCliente"] == df_clientes["IDCliente"], "left_outer") \
    .withColumn("IntegridadeReferencial",
                when(df_clientes["IDCliente"].isNotNull(), lit("Válido"))
                .otherwise(lit("Inválido")))

# Mostrando o DataFrame resultante
df_verificado.show()

+---------------+---------+-----------+---------+----------------------+
|DescricaoPedido|IDCliente|NomeCliente|IDCliente|IntegridadeReferencial|
+---------------+---------+-----------+---------+----------------------+
|        Pedido1|        1|   Cliente1|        1|                Válido|
|        Pedido2|        2|   Cliente2|        2|                Válido|
|        Pedido3|        4|       NULL|     NULL|              Inválido|
+---------------+---------+-----------+---------+----------------------+



In [ ]:
spark.stop()